In [1]:
## THIS CELL SHOULD BE IN ALL VSCODE NOTEBOOKS ##

MARKET = 'NSE'

from loguru import logger

import pandas as pd
pd.options.display.max_columns=None

# Add `src` to _src.pth in .venv to allow imports in VS Code
from sysconfig import get_path
from pathlib import Path
if 'src' not in Path.cwd().parts:
    src_path = str(Path(get_path('purelib')) / '_src.pth')
    with open(src_path, 'w') as f:
        f.write(str(Path.cwd() / 'src\n'))

# Start the Jupyter loop
from ib_async import util
util.startLoop()

# Imports 
from utils import Vars, get_pickle

# Set the root
from from_root import from_root
ROOT = from_root()

# Scrape lots and expiries from Zerodha

In [1]:
from utils import zerodha_lots_expiries
df = zerodha_lots_expiries()

# Scrape margins from Zerodha's margin calculator

In [3]:
data = {"action": "calculate",
        "exchange%5B%5D": "NFO",
        "product%5B%5D": "OPT",
        "scrip%5B%5D": "AARTIIND24JUL",
        "option_type%5B%5D": "PE",
        "strike_price%5B%5D": 660,
        "qty%5B%5D": 1000,
        "trade%5B%5D": "sell"}

In [ ]:
from utils import get_zerodha_span_payload
